#### Copyright 2019 Google LLC.

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Video Processing

We will soon be working on a project that processes video and attempts to classify objects in that video. To prepare for the project we'll do a quick Colab that processes a video.

## Overview

### Learning Objectives

* Use OpenCV to process video

### Prerequisites

* Intermediate Python

### Estimated Duration

30 minutes

### Grading Criteria

Each exercise is worth 3 points. The rubric for calculating those points is:

| Points | Description |
|--------|-------------|
| 0      | No attempt at exercise |
| 1      | Attempted exercise, but code does not run |
| 2      | Attempted exercise, code runs, but produces incorrect answer |
| 3      | Exercise completed successfully |

There is 1 exercise in this Colab so there are 3 points available. The grading scale will be 3 points.

## Obtain a Video

Start by uploading the smallest version of [this video](https://pixabay.com/videos/cars-motorway-speed-motion-traffic-1900/) to the colab. Rename the video to "cars.mp4" or change the name of the video in the code below.

## Reading Video

OpenCV is an Open Source library for performing computer vision tasks. One of these tasks is reading and writing video frames. To read a video file we use the [VideoCapture](https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html#videocapture) class.

In [2]:
import cv2 as cv

cars_video = cv.VideoCapture('cars.mp4')

Once you have created a `VideoCapture` object you can obtain information about the video that you are processing.

In [3]:
height = int(cars_video.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(cars_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = cars_video.get(cv.CAP_PROP_FPS)
total_frames = int(cars_video.get(cv.CAP_PROP_FRAME_COUNT))

print(f'height: {height}')
print(f'width: {width}')
print(f'frames per second: {fps}')
print(f'total frames: {total_frames}')
print(f'video length (seconds): {total_frames / fps}')

height: 720
width: 1280
frames per second: 50.0
total frames: 3000
video length (seconds): 60.0


Once you are done, it is a good idea to release the video to free up memory in your program.

In [4]:
cars_video.release()

You can loop through the video frame-by-frame.
 
To do this you need to know the total number of frames in the video. For each frame you set the current frame position and then read that frame. This causes the frame to be loaded from disk into memory. This is done because videos can be huge so you don't necessarily want the entire thing in memory.
 
You might also notice that we read the frame from the cars video and then check the return value to make sure that the read was successful. This is because the underlying video processing library is written in the C++ program language. A common practice in that language is to return a status code indicating if a function succeed or not. This isn't a very idiomatic Python, it is just the underlying libraries style leaking through into the Python wrapper.

In [ ]:
cars_video = cv.VideoCapture('cars.mp4')

total_frames = int(cars_video.get(cv.CAP_PROP_FRAME_COUNT))

frames_read = 0

for current_frame in range(0, total_frames):
  cars_video.set(cv.CAP_PROP_POS_FRAMES, current_frame)
  ret, _ = cars_video.read()
  if not ret:
    raise Exception(f'Problem reading frame {current_frame} from video')
  if (current_frame+1) % 50 == 0:
    print(f'Read {current_frame+1} frames so far')

cars_video.release()

print(f'Read {total_frames} frames')

That was pretty slow. The video is just over a minute long and it takes a while to iterate over every frame. Imagine the amount of time it would take to perform object recognition on each frame.

In practice you'd be doing this processing on a much bigger machine (or machines) than Colab provides for free. You can also process many frames in parallel.

We'll resort to a different tactic for this lab though. Let's just make the video shorter.

We'll load the video one more time and read out a single frame to illustrate that the frame is just an image.

In [ ]:
import matplotlib.pyplot as plt

cars_video = cv.VideoCapture('cars.mp4')
cars_video.set(cv.CAP_PROP_POS_FRAMES, 123)
ret, frame = cars_video.read()
if not ret:
  raise Exception(f'Problem reading frame {current_frame} from video')

cars_video.release()

plt.imshow(frame)

## Writing Video

OpenCV also supports writing video data. Let's loop through the long video that we have and only save one second of it into a new file.

First we need to open our input video and get information about the frame rate, height, and width.

In [ ]:
input_video = cv.VideoCapture('cars.mp4')

height = int(input_video.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(input_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = input_video.get(cv.CAP_PROP_FPS)

Using that information we can then create a [VideoWriter](https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html#videowriter) that we'll use to write the shorter video.

Video can be encoded using many different formats. In order to tell OpenCV which format to use we choose a "four character code" from [fourcc](https://www.fourcc.org/). In this case we use "mp4v" to keep our input and output files consistent.

In [ ]:
fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_video = cv.VideoWriter('cars-short.mp4', fourcc, fps, (width, height))

---
 
>  As an aside, what is that star doing in front of the 'mp4v' string?
 
>  It is performing variable unpacking.
 
>  Since a string is an iterable it unpacks each letter of the string into a separator argument.
 
 >  > *'mp4v' becomes 'm', 'p', '4', 'v'
 
> You can pass either the separate characters or the starred-string to `VideoWriter_fourcc`.
 
---


Now we can loop through the first one second of video frames and write each frame to our output video.

In [ ]:
for i in range(0, int(fps)):
  input_video.set(cv.CAP_PROP_POS_FRAMES, i)
  ret, frame = input_video.read()
  if not ret:
    raise Exception("Problem reading frame", i, " from video")
  output_video.write(frame)

Once processing is complete, but sure to release the video objects.

In [ ]:
input_video.release()
output_video.release()

And now we can list the directory to see if our new file was created.

In [ ]:
import os

os.listdir('./')

You should now see a 'cars-short.mp4' file in your file browser in Colab. Download and view the video to make sure that it only lasts for a second.

Notice that we have only concerned ourselves with the visual portion of the video. Videos contain both visual and auditory elements. OpenCV is only concerned with computer vision, so it doesn't handle audio processing.

# Exercises

## Exercise 1

In the Colab above we shortened our video to 1-second be simply grabbing the first second of frames from the video file. Since not much typically changes from frame-to-frame within a second of video, a better video processing technique is to sample frames throughout the entire video and skip some frames. For example, it might be more beneficial to process every 10th frame or only process 1 of the frames in every second of video.

In this exercise take the original cars video used in this colab and reduce it to a short 25-fps video by grabbing the first frame of every second of video. Save the video as 'cars-sampled.mp4'.

### Student Solution

In [ ]:
# Your code goes here
input_video = cv.VideoCapture('cars.mp4')

height = int(input_video.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(input_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = input_video.get(cv.CAP_PROP_FPS)
totalframe =  input_video.get(cv.CAP_PROP_FRAME_COUNT)

fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_video = cv.VideoWriter('cars-sampled.mp4', fourcc, fps, (width, height))

for i in range(0, int(totalframe),int(fps)):
  input_video.set(cv.CAP_PROP_POS_FRAMES, i)
  ret, frame = input_video.read()
  if not ret:
    raise Exception("Problem reading frame", i, " from video")
  output_video.write(frame)

input_video.release()
output_video.release()

##Exercise 2

Make a second copy of the video you made in the previous exercise. In this copy, the frames run in reverse, so everything appears to move backwards.

In [ ]:
# Your code goes here
input_video = cv.VideoCapture('cars-sampled.mp4')

height = int(input_video.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(input_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = input_video.get(cv.CAP_PROP_FPS)
totalframe =  input_video.get(cv.CAP_PROP_FRAME_COUNT)

fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_video = cv.VideoWriter('cars-sampled-reverse.mp4', fourcc, fps, (width, height))

for i in range(int(totalframe)-1,-1,-1):
  input_video.set(cv.CAP_PROP_POS_FRAMES, i)
  ret, frame = input_video.read()
  if not ret:
    raise Exception("Problem reading frame", i, " from video")
  output_video.write(frame)

input_video.release()
output_video.release()

##Exercise 3

Make a third copy of the video you made in the first exercise. In this copy, manipulate each frame to remove the colors and create a "black and white" (greyscale) video.

In [ ]:
# Your code goes here
input_video = cv.VideoCapture('cars-sampled.mp4')

height = int(input_video.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(input_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = input_video.get(cv.CAP_PROP_FPS)
totalframe =  input_video.get(cv.CAP_PROP_FRAME_COUNT)

fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_video = cv.VideoWriter('cars-sampled-greyscale.mp4', fourcc, fps, (width, height),isColor = False)

for i in range(0, int(totalframe)):
  input_video.set(cv.CAP_PROP_POS_FRAMES, i)
  ret, frame = input_video.read()
  grayframe = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
  if not ret:
    raise Exception("Problem reading frame", i, " from video")
  output_video.write(grayframe)

input_video.release()
output_video.release()